## Import Modules

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from astropy.io import fits
import colorsys

## Save .fits files as Numpy arrays

In [ ]:
#import 1st image
fname1 = "f105w.fits"

#open using astropy.io
f105 = fits.open(fname1)
#save as 2d numpy array
d1_data = f105[0].data

#close using astropy.io
f105.close()

#import 2nd image
fname2 = "f125w.fits"

#open using astropy.io
f125 = fits.open(fname2)
#save as 2d numpy array
d2_data = f125[0].data

#close using astropy.io
f125.close()

#import 3rd image
fname3 = "f160w.fits"

#open using astropy.io
f160 = fits.open(fname3)
#save as 2d numpy array
d3_data = f160[0].data

#close using astropy.io
f160.close()

## Define a funct to adjust images

In [ ]:
def adjust(image,cut_fac,log_fac):
    
    #we want to ignore the large ammount of gray 
    #around the sides of the image
    
    #to get more accurate values for the mean and stdev
    #we crop the image
    crop = image[1500:2000,1500:2000]
    mean = np.mean(crop)
    stdev = np.std(crop)
    
    #values that are many stdevs away from the mean will cause
    #the vast majority of pixels to look the same
    
    #we set values above this limit to 
    #instead be equal to the limit itself
    up_limit  = mean + cut_fac*stdev

    for i in range(3600):
        for j in range(3600):
            if image[i,j] > up_limit:
                image[i,j] = up_limit
                
    #in order to take the log of the image,
    #we must make sure all negative and zero values
    #become positive
    
    #subtracting minimum to all image values
    #makes min equal zero
    
    #add small extra value so all values are pos
    
    min_image = np.min(image)
    adj = image - min_image + log_fac
                
    im_log = np.log(adj)
    
    #adjust array so that the new minimum is 0 and max is 1
    log_min = np.min(im_log)
    im_pos = im_log - log_min
    
    pos_max = np.max(im_pos)
    im_final = im_pos / pos_max
                
    return im_final

## Adjust and plot 1st Figure

In [ ]:
#use values cut_fac = 10 and log_fac = 0.05 
d1_adjust = adjust(d1_data,10,0.05)

f = plt.figure(figsize=(10,10))
plt.imshow(d1_adjust, cmap='gray')
plt.colorbar()

## Adjust and plot 2nd figure

In [ ]:
#use values cut_fac = 10 and log_fac = 0.05 
d2_adjust = adjust(d2_data,10,0.05)

f = plt.figure(figsize=(10,10))
plt.imshow(d2_adjust, cmap='gray')
plt.colorbar()

## Adjust and plot 3rd figure

In [ ]:
#use values cut_fac = 10 and log_fac = 0.05 
d3_adjust = adjust(d3_data,10,0.05)

f = plt.figure(figsize=(10,10))
plt.imshow(d3_adjust, cmap='gray')
plt.colorbar()

## Combine Images into one RGB image and plot

In [ ]:
#create blank canvas with 3 clannels for every pixel
RGB = np.zeros((3600,3600,3))
HSV = np.zeros((3600,3600,3))

#1st channel = red, set to 1st image
RGB[:,:,0] = d1_adjust[:,:]

#2nd channel = green, set to 2nd image
RGB[:,:,1] = d2_adjust[:,:]

#3rd channel = blue, set to 3rd image
RGB[:,:,2] = d3_adjust[:,:]

#plot figure
f = plt.figure(figsize=(10,10))
plt.imshow(RGB)
f.savefig('rgb_hudf.png')